In [48]:
# pip3.6 install wheel
# pip3.6 install requests
# pip3.6 install psycopg2
# pip3.6 install pandas
# pip3.6 install ontobio
# pip3.6 install pymongo
import os
os.chdir('/Volumes/DATADRIVE/GIT/VFB_neo4j/src')
from uk.ac.ebi.vfb.neo4j.KB_tools import KB_pattern_writer
import pandas as pd
from ontobio import OntologyFactory
import re
import yaml
kbw = KB_pattern_writer('http://localhost:7474', 'neo4j', 'password')
OF = OntologyFactory()
fbbt = OF.create("/Volumes/DATADRIVE/GIT/drosophila-anatomy-developmental-ontology/fbbt/releases/fbbt.json")
mapping_table = pd.read_csv("/Volumes/DATADRIVE/GIT/DrosAdultVNSdomains/refData/IndexKey_Live.tsv", sep='\t')
mapping_table

/Volumes/DATADRIVE/GIT/VFB_neo4j/src/uk/ac/ebi/vfb/neo4j/KB_tools.py:132: UserWarning: No existing ids match the pattern VFB_nnnnnnnn
  warnings.warn("No existing ids match the pattern %s_%s" % (idp, 'n'*acc_length))
/Volumes/DATADRIVE/GIT/VFB_neo4j/src/uk/ac/ebi/vfb/neo4j/KB_tools.py:132: UserWarning: No existing ids match the pattern VFBc_nnnnnnnn
  warnings.warn("No existing ids match the pattern %s_%s" % (idp, 'n'*acc_length))


,Index,Ind_ID,Class_ID,Name,Abbreviation,Note,Centre
0,0,VFB_00100000,FBbt_00004052,Ventral nervious system,VNS,NaN,"[258,485,110]"
1,1,VFB_00100001,FBbt_00110176,Metathoracic Neuromere,MetaNm,(sensory area 26),"[257,656,109]"
2,2,VFB_00100002,FBbt_00110175,Mesothoracic Neuromere,MesoNm,(sensory area 24),"[257,429,87]"
3,3,VFB_00100003,FBbt_00110174,Prothoracic Neuromere,ProNm,(sensory area 25),"[258,176,113]"
4,4,VFB_00100004,FBbt_00004019,Cervical Connective,CvC,NaN,"[243,73,154]"
5,5,VFB_00100005,FBbt_00110173,Abdominal ganglionic complex,AGC,(sensory area 27),"[253,819,121]"
6,6,VFB_00100006,FBbt_00004099,Abdominal nerve trunk,AbNT,MAN,"[221,964,103]"
7,7,VFB_00100007,FBbt_00004108,Forth abdominal nerve,AbN4,SLA,"[225,945,117]"
8,8,VFB_00100008,FBbt_00004107,Third abdominal nerve,AbN3,FLA,NaN
9,9,VFB_00100009,FBbt_00004106,Second abdominal nerve,AbN2,EMN,"[223,828,162]"


In [63]:
import math
ltd = []
for i,r in mapping_table.iterrows():
    LR = ''
    if re.match('.+Left', r.Name): LR = "left "
    if re.match('.+Right', r.Name): LR = "right "
    class_name = fbbt.label(re.sub('_', ':', r.Class_ID))
    if class_name is None:
        label_core = str(r.Name).lower()
        break
    else:
        label_core = str(class_name)
    d = {}
    d['label'] = label_core + ' on adult VNS template, Court2018' 
    d['synonyms'] = [r.Abbreviation, str(r.Name).lower()]
    d['Index'] = r.Index
    d['Anatomical_type'] = r.Class_ID
    if isinstance(r.Centre, str):
        d['centre'] = r.Centre
    else:
        d['centre'] = [258,485,110]
    d['VFB_id'] = r.Ind_ID
    ltd.append(d)
    
loading_table = pd.DataFrame.from_records(ltd)
loading_table

,Anatomical_type,Index,VFB_id,centre,label,synonyms
0,FBbt_00004052,0,VFB_00100000,"[258,485,110]",thoracico-abdominal ganglion on adult VNS temp...,"[VNS, ventral nervious system]"
1,FBbt_00110176,1,VFB_00100001,"[257,656,109]",adult metathoracic neuromere on adult VNS temp...,"[MetaNm, metathoracic neuromere ]"
2,FBbt_00110175,2,VFB_00100002,"[257,429,87]",adult mesothoracic neuromere on adult VNS temp...,"[MesoNm, mesothoracic neuromere ]"
3,FBbt_00110174,3,VFB_00100003,"[258,176,113]",adult prothoracic neuromere on adult VNS templ...,"[ProNm, prothoracic neuromere ]"
4,FBbt_00004019,4,VFB_00100004,"[243,73,154]","cervical connective on adult VNS template, Cou...","[CvC, cervical connective]"
5,FBbt_00110173,5,VFB_00100005,"[253,819,121]",adult abdominal neuromere on adult VNS templat...,"[AGC, abdominal ganglionic complex]"
6,FBbt_00004099,6,VFB_00100006,"[221,964,103]","abdominal nerve trunk on adult VNS template, C...","[AbNT, abdominal nerve trunk]"
7,FBbt_00004108,7,VFB_00100007,"[225,945,117]",abdominal segment 4 nerve on adult VNS templat...,"[AbN4, forth abdominal nerve]"
8,FBbt_00004107,8,VFB_00100008,"[258, 485, 110]",abdominal segment 3 nerve on adult VNS templat...,"[AbN3, third abdominal nerve]"
9,FBbt_00004106,9,VFB_00100009,"[223,828,162]",abdominal segment 2 nerve on adult VNS templat...,"[AbN2, second abdominal nerve]"


In [64]:
lt_out = loading_table.copy()

for k,r in lt_out.iterrows():
    synstring = "|".join(r.synonyms)
    lt_out[k, "synonyms"] = synstring
    cstring = "|".join([str(x) for x in r.centre])
    lt_out.at[k, "centre"] = cstring

lt_out.to_csv("/Volumes/DATADRIVE/GIT/DrosAdultVNSdomains/refData/loading_table.tsv", sep = "\t")

In [66]:
for i,r in loading_table.iterrows():
    anatomy_attributes = {}
    anatomy_attributes['synonyms'] = r.synonyms
    iris = kbw.add_anatomy_image_set(dataset = "Court2018",
                                     index = r.Index,
                                     imaging_type = "computer graphic", 
                                     label = r.label,
                                     start = 50002, 
                                     template = 'VFBc_00100000', 
                                     anatomical_type=r.Anatomical_type,
                                     anatomy_attributes=anatomy_attributes,
                                     center=r.centre       
                                     )
    r.VFB_id = iris['anatomy']
loading_table.to_csv("/Volumes/DATADRIVE/GIT/DrosAdultVNSdomains/refData/Domains_report.tsv", sep = "\t")